# Generate homography mapping for Crichton camera installation 

Pair up keypoints from each camera to a farm position to generate a mapping between each camera and a global coordinate system.

Below is a sample of the top-down mapping for a few of the cameras on the installation, showing areas with an overlap and how they interact.

For convenience, the regularly-spaced farm landmarks are automatically converted from a `row,pillar` annotation to a global coordinate,

- `(row=0, pillar=0)` is equivalent to global `(0, 0)`
- `(row=2, pillar=7)` is equivalent to global `(120, 360)`

The maximum dimension `(120, 360)` was chosen as this farm shed is `12m` by `36m`, so the global pixel coordinate is easily mentally translatable to a farm position.

In [ ]:
print(open("farm-layout.txt").read())

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage import transform
import homography as h
from imgutil import crop_and_pad, plot_points_as_box, loadrgba, remove_box, remove_ticks

## Pair up each image


# `def` warp_using_points

In [ ]:
def transform_image(image, matrix):
    # Using scikit-image, warp the image using the matrix
    # we need to add a slight border to the image to allow for edge areas to get transformed

    image_new = transform.warp(image, matrix.inverse)

    # Crop the image to the expected dimensions,
    # so points that get transformed off the edge (as we are mapping floor area)
    # do not extend the boundaries of the image
    image_new = crop_and_pad(image_new, h.FARM_DIM)
    return image_new

# `def` Function which applies a homography mapping to a single coordinate point

In [ ]:
def perspective_warp_coord(point, matrix):
    """Warp a single (x,y) point using a homography matrix.

    This uses matrix multiplication to warp a 2D point.
    - A z value of 1 is added to facilitate the 3D transform;
    - this is then used to calculate the norm of the vector (z * [x; y; z]).
    - The point is then multiplied by the matrix, and normalised using this.
    - Finally, we slice the matrix to only return the warped [x, y] point.

    This uses the '@' operator, introduced in Python 3.5, which calls
    __matmul__ on the object.

    Arguments
    ---------
    point : Tuple[int, int]
    matrix : np.array

    Returns
    -------
    warped_point : Tuple[int, int]
    """
    point = np.array([*point, 1])  # [x, y, 1]
    norm = matrix[2] @ point
    return np.ceil(matrix @ point / norm)[:2]


# `def` helper to generate transform

In [ ]:
def warp_using_points(filename, keypoints):
    assert h.FARM_DIM
    # Read in the image and correct the colour channel order
    # as opencv loads BGR but matplotlib expects RGB
    im = loadrgba(filename)

    # Pillars in the farm are 5.25 metres apart, and the central reserve is
    # 6m across the 12m shed
    # Convert the pillar and row number into metres, and scale up to the desired
    # farm dimension
    keypoints_cam = []
    keypoints_top = []
    pix_w, pix_h = h.FARM_DIM
    farm_w, farm_h = 7 * 5.25, 12

    for kp in keypoints:
        keypoints_cam.append(kp["camera"])
        xtop = kp["pillar"] * 5.25 / farm_w * pix_w
        ytop = kp["row"] * 6 / farm_h * pix_h
        keypoints_top.append((xtop, ytop))

    # Estimate a transform that would match each keypoint from the camera view
    # onto the top-down global perspective
    keypoints_cam = np.array(keypoints_cam)
    keypoints_top = np.array(keypoints_top)
    matrix = transform.estimate_transform("projective", keypoints_cam, keypoints_top)

    im_top = transform_image(im, matrix)

    # -----------------------------------------------------------------------
    #        Below is utility stuff for plotting and calculating error
    # -----------------------------------------------------------------------
    fs = 15
    _, (axcam, axtop) = plt.subplots(nrows=2, figsize=(8, 8))

    axcam.imshow(cv2.convertScaleAbs(im, alpha=1.4, beta=30))
    axcam.grid()
    axcam.scatter(keypoints_cam[:, 0], keypoints_cam[:, 1], c="r", s=10)
    for i, (x, y) in enumerate(keypoints_cam):
        axcam.text(x * 1.01, y * 1.01, str(i + 1), fontsize=fs, c="red")

    axtop.imshow(im_top)
    axtop.scatter(keypoints_top[:, 0], keypoints_top[:, 1], c="r")
    for i, (x, y) in enumerate(keypoints_top):
        axtop.text(x * 1.01, y * 1.01, str(i + 1), fontsize=fs, c="r")
    axtop.set_xlim(0, 52.5*7)
    axtop.set_ylim(120, 0)
    plt.tight_layout(h_pad=0, w_pad=0)
    plt.grid(alpha=0.3)

    # Calculate the error that happens during transformation by comparing
    # the converted pixel coordinate to the target coordinate
    #
    pixel_x_err, pixel_y_err = [], []
    for kp, kpt in zip(keypoints_cam, keypoints_top):
        kpct = perspective_warp_coord(kp, matrix.params)
        pixel_x_err.append(kpct[0] - kpt[0])
        pixel_y_err.append(kpct[1] - kpt[1])
    pixel_x_mean_err = np.mean(pixel_x_err)
    pixel_x_std_err = np.std(pixel_x_err)
    pixel_y_mean_err = np.mean(pixel_y_err)
    pixel_y_std_err = np.std(pixel_y_err)

    print(f"Homography keypoint error, in pixels:")
    print(f"\tx {pixel_x_mean_err:.1f} mean {pixel_x_std_err:.1f} std")
    print(f"\ty {pixel_y_mean_err:.1f} mean {pixel_y_std_err:.1f} std")

    return matrix

# Generate top-down mapping

For each camera, we pass a list of `camera, row, pillar` dictionary items.

| Key | Type | Description |
|:--|:--|:--|
| `camera` | `tuple[int, int]` | `(x, y)` image coordinate matching a farm landmark |
| `pillar` | `int` in range `0..7` | numbered with `0` as camera 14 in the above diagram, and `7` aligned with camera 7 |
| `row` | `int` in range `0..2` | indicating `0 right`, `1: middle`, `2: left`, as per the diagram above |

## 01 left

In [ ]:
cam = "01left"
matrix_1left = warp_using_points(
    f"img/{cam}.png",
    keypoints=[
        {'camera': (282, 312), 'row':0, 'pillar':1},
        {'camera': (407, 190), 'row':0, 'pillar': 2},
        {'camera': (466, 129), 'row':0, 'pillar': 3},
        {'camera': (500, 96), 'row':0, 'pillar': 4},
        {'camera': (520, 75), 'row':0, 'pillar': 5},
        {'camera': (539, 49), 'row':0, 'pillar': 6.8},
        {'camera': (630, 294), 'row':1, 'pillar':1},
        {'camera': (620, 37), 'row':1, 'pillar': 6.8},
    ]
)


## 06 right

In [ ]:
cam = "06right"
matrix_6right = warp_using_points(
    f"img/{cam}.png",
    keypoints=[
        {'camera': (75, 142), 'row': 0, 'pillar': 2.5},
        {'camera': (170, 148), 'row': 0, 'pillar': 3},
        {'camera': (255, 148), 'row': 0, 'pillar': 3.5},
        {'camera': (383, 157), 'row': 0, 'pillar': 4.5},
        {'camera': (434, 159), 'row': 0, 'pillar': 5},
        {'camera': (543, 171), 'row': 0, 'pillar': 6.8},
        {'camera': (135, 287), 'row': 1, 'pillar': 2.5},
        {'camera': (271, 281), 'row': 1, 'pillar': 3},
        {'camera': (464, 258), 'row': 1, 'pillar': 4},
        {'camera': (550, 230), 'row': 1, 'pillar': 5},
        {'camera': (599, 225), 'row': 1, 'pillar': 6},
        {'camera': (623, 217), 'row': 1, 'pillar': 6.8},
    ],
)
# del_bad_cam(cam)


## 13 right


In [ ]:
cam = "13right"
matrix_13right = warp_using_points(
    f"img/{cam}.png",
    keypoints=[
        {'camera': (622, 332), 'row': 0, 'pillar': 3},
        {'camera': (533, 164), 'row': 1, 'pillar': 2},
        {'camera': (434, 159), 'row': 1, 'pillar': 3},
        {'camera': (207, 151), 'row': 1, 'pillar': 4},
        {'camera': (550, 155), 'row': 1.2, 'pillar': 1.2},
        {'camera': (525, 135), 'row': 1.4, 'pillar': 1.3},
        {'camera': (544, 104), 'row': 2, 'pillar': 0},
        {'camera': (424, 75), 'row': 2, 'pillar': 2},
    ]
)
# del_bad_cam(cam)


## 15 left


In [ ]:
cam = "15left"
matrix_15left = warp_using_points(
    f"img/{cam}.png",
    keypoints=[
        {'camera': (46, 329), 'row': 0, 'pillar': 4},
        {'camera': (495, 152), 'row': 1, 'pillar': 2.5},
        {'camera': (350, 157), 'row': 1, 'pillar': 3},
        {'camera': (186, 161), 'row': 1, 'pillar': 4},
        {'camera': (108, 164), 'row': 1, 'pillar': 5},
        {'camera': (58, 165), 'row': 1, 'pillar': 6},
        {'camera': (37, 163), 'row': 1, 'pillar': 6.8},
        {'camera': (111, 101), 'row': 2, 'pillar': 6.8},
    ],
)


## Show transfer of a bounding box from camera to global view

In [ ]:
def box_from_xywh(x, y, w, h):
    # utility to convert xywh to a list of corner coordinates
    return [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]

In [ ]:
cam_img = loadrgba("img/15left.png")
mat = matrix_15left

f, axes = plt.subplots(2, figsize=(8, 8), tight_layout=True)

# Show an image of the camera, and a bounding box identifying one cow
axes[0].imshow(cam_img)
bb = box_from_xywh(190, 50, 90, 70)
plot_points_as_box(bb, axes[0], color='red')

# Transform the camera to it's top-down (global) view
# and also transform each corner of the bounding box
cam_top_img = transform_image(cam_img, mat)
axes[1].imshow(cam_top_img)
bbtop = [h.perspective_warp_coord(corner, mat.params) for corner in bb]
plot_points_as_box(bbtop, axes[1], color='red')

# Remove axes and tidy up the plot
remove_box(axes[0])
remove_box(axes[1])
remove_ticks(axes[0])
remove_ticks(axes[1])
axes[1].set_xlim(0, 360)
axes[1].set_ylim(120, 0)

# Transfer bounding box between views

Now we have a global coordinate system, we can either transform points from each camera onto the top as a shared reference, or use the top view as an intermediate to transform points between each camera.

We have an overlapping camera view betwen `06right` and `15left`, so we will use them as an example.

## From each camera onto top 

In [ ]:
cam_img = loadrgba("img/15left.png")
cam2_img = loadrgba("img/06right.png")

mat = matrix_15left
mat2 = matrix_6right

f = plt.figure(figsize=(8, 8), tight_layout=True)
axTL = plt.subplot2grid((3, 2), (0, 0))
axTR = plt.subplot2grid((3, 2), (0, 1))
axML = plt.subplot2grid((3, 2), (1, 0))
axMR = plt.subplot2grid((3, 2), (1, 1))
axBOT = plt.subplot2grid((3, 2), (2, 0), colspan=2)

# Show an image of the camera, and a bounding box identifying one cow
axTL.imshow(cam_img)
bb = box_from_xywh(190, 50, 90, 70)
plot_points_as_box(bb, axTL, color='red')

axTR.imshow(cam2_img)
bb2 = box_from_xywh(540, 240, 60, 100)
plot_points_as_box(bb2, axTR, color='blue')

# Transform the camera to it's top-down (global) view
# and also transform each corner of the bounding box
cam_top_img = transform_image(cam_img, mat)
cam2_top_img = transform_image(cam2_img, mat2)
axML.imshow(cam_top_img)
axMR.imshow(cam2_top_img)
bbtop = [perspective_warp_coord(corner, mat.params) for corner in bb]
bb2top = [perspective_warp_coord(corner, mat2.params) for corner in bb2]
plot_points_as_box(bbtop, axML, color='red')
plot_points_as_box(bb2top, axMR, color='blue')

# Generate a transparent region showing the floormap of both cameras
top_1 = np.zeros((120, 360, 4)).astype(int)
xs, ys = np.where(cam_top_img[:, :, 3].astype(int))
print(xs, ys)
top_1[~xs, ~ys, 3] = 0
top_1[xs, ys, 3] = 20
top_1[xs, ys, 0] = 255

top_2 = np.zeros((120, 360, 4)).astype(int)
xs, ys = np.where(cam2_top_img[:, :, 3].astype(int))
top_2[~xs, ~ys, 3] = 0
top_2[xs, ys, 3] = 20
top_2[xs, ys, 2] = 255

axBOT.imshow(top_1.astype(int))
axBOT.imshow(top_2.astype(int))
plot_points_as_box(bbtop, axBOT, color='red')
plot_points_as_box(bb2top, axBOT, color='blue')


# Remove axes and tidy up the plot
for ax in [axTL, axTR, axML, axMR, axBOT]:
    remove_ticks(ax)

for ax in [axML, axMR, axBOT]:
    ax.set_xlim(0, 360)
    ax.set_ylim(120, 0)

## From one camera to another

In [ ]:
cam_img = loadrgba("img/15left.png")
cam2_img = loadrgba("img/06right.png")

mat = matrix_15left
mat2 = matrix_6right

f, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8, 8), tight_layout=True)

# Show an image of the camera, and a bounding box identifying one cow
ax1.imshow(cam_img)
bb = box_from_xywh(250, 50, 60, 90)
plot_points_as_box(bb, ax1, color='red')

ax2.imshow(cam2_img)
bb2 = box_from_xywh(420, 230, 130, 100)
plot_points_as_box(bb2, ax2, color='blue')

bbtop = [perspective_warp_coord(corner, mat.params) for corner in bb]
bb2top = [perspective_warp_coord(corner, mat2.params) for corner in bb2]

bb_to_bb2 = [perspective_warp_coord(corner, np.linalg.inv(mat2)) for corner in bbtop]
bb2_to_bb = [perspective_warp_coord(corner, np.linalg.inv(mat)) for corner in bb2top]

plot_points_as_box(bb2_to_bb, ax1, color='blue')
plot_points_as_box(bb_to_bb2, ax2, color='red')

# Remove axes and tidy up the plot
for ax in [ax1, ax2]:
    remove_ticks(ax)
    ax.set_xlim(0, 640)
    ax.set_ylim(360, 0)

ax1.set_title('Red bounding box, and converted-from-view-2 Blue bounding box')
ax2.set_title('Blue bounding box, and converted-from-view-1 Red bounding box')